# テストの実行

In [35]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [36]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work', '03_connection_set')

## マスタの読み込み

In [37]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_test_pre.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_test_sys.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_test_stress.csv'), usecols=['seed'])

## 実行するロジックの指定

In [38]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'sa_base_1k'

In [39]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [40]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=9) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(D)
        solve_result.append(K)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # 非連結になった日数
            discon_days = int(res.stderr.decode('utf-8').split()[0].replace('DisConDay=', ''))

            # 工事可能日数ごとの辺数
            avail_0_cnt = int(res.stderr.decode('utf-8').split()[1].replace('Avail0=', ''))
            avail_1_cnt = int(res.stderr.decode('utf-8').split()[2].replace('Avail1=', ''))
            avail_2_cnt = int(res.stderr.decode('utf-8').split()[3].replace('Avail2=', ''))
            avail_3_cnt = int(res.stderr.decode('utf-8').split()[4].replace('Avail3=', ''))

            schedule_room = float(res.stderr.decode('utf-8').split()[5].replace('ScheduleRoom=', ''))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(discon_days)
        solve_result.append(avail_0_cnt)
        solve_result.append(avail_1_cnt)
        solve_result.append(avail_2_cnt)
        solve_result.append(avail_3_cnt)
        
        solve_result.append(schedule_room)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'discon_days', 'avail_0_cnt', 'avail_1_cnt', 'avail_2_cnt', 'avail_3_cnt', 'schedule_room']

    return result_df

In [48]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [54]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'discon_days_mean': [(np.mean(test_result_df['discon_days']))],
            'discon_days_min': [min(test_result_df['discon_days'])],
            'discon_days_max': [max(test_result_df['discon_days'])],

            'avail_0_cnt_mean': [np.mean(test_result_df['avail_0_cnt'])],
            'avail_0_cnt_max': [max(test_result_df['avail_0_cnt'])],

            'avail_1_cnt_mean': [np.mean(test_result_df['avail_1_cnt'])],
            'avail_2_cnt_mean': [np.mean(test_result_df['avail_2_cnt'])],
            'avail_3_cnt_mean': [np.mean(test_result_df['avail_3_cnt'])],

            'schedule_room_mean': [np.mean(test_result_df['schedule_room'])],
            'schedule_room_min': [min(test_result_df['schedule_room'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    summary_all_df = summary_all_df[['tag', 'testset', 'time_mean', 
                                     'discon_days_mean', 'discon_days_min', 'discon_days_max',
                                     'avail_0_cnt_mean', 'avail_0_cnt_max',
                                     'avail_1_cnt_mean', 'avail_2_cnt_mean', 'avail_3_cnt_mean',
                                     'schedule_room_mean', 'schedule_room_min',
                                     'time_max']]
    
    return summary_all_df

In [55]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#testset_name = '00_test_mini'
#testset_name = '01_test_pre'
#testset_name = '02_test_sys'
testset_name = '03_test_stress'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-01-29 15:35:11,723: Start
2023-01-29 15:38:05,521: finish!


In [56]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,discon_days_mean,discon_days_min,discon_days_max,avail_0_cnt_mean,avail_0_cnt_max,avail_1_cnt_mean,avail_2_cnt_mean,avail_3_cnt_mean,schedule_room_mean,schedule_room_min,time_max
0,sa_base_1k,01_test_pre,164,0.0,0,0,0.0,0,11.28,33.7,62.18,10.785462,3.23881,311


In [57]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_test_sys' or testset_name == '03_test_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,discon_days_mean,discon_days_min,discon_days_max,avail_0_cnt_mean,avail_0_cnt_max,avail_1_cnt_mean,avail_2_cnt_mean,avail_3_cnt_mean,schedule_room_mean,schedule_room_min,time_max
0,sa_base_1k,02_test_sys,149,0.0,0,0,0.0,0,27.4715,57.428,92.2515,9.083988,1.95137,344


In [58]:
# stress test
summary_df = pd.DataFrame()

if testset_name == '03_test_stress':
    result_sub_df = pd.merge(stress_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '03_test_stress'

summary_df

,tag,testset,time_mean,discon_days_mean,discon_days_min,discon_days_max,avail_0_cnt_mean,avail_0_cnt_max,avail_1_cnt_mean,avail_2_cnt_mean,avail_3_cnt_mean,schedule_room_mean,schedule_room_min,time_max
0,sa_base_1k,03_test_stress,149,0.0,0,0,0.0,0,26.5303,56.4204,91.5712,9.091341,1.875,369


# 結果ログの保存

In [51]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)